In [1]:
import pandas as pd

In [2]:
def long_stalking(old_stalking_file, marketsmith_file, new_stalking_file):
    long_file = pd.read_excel(f"C:/Users/krish/Trading Dropbox/Krishna Chaitanya/Process/Stalking/DailyStalking/"
                              f"2024/05.May/Long/{old_stalking_file}.xlsx")
    market_smith_file = pd.read_excel(f"C:/Users/krish/Downloads/{marketsmith_file}.xlsx")

    for ticker in range(len(long_file["Ticker"])):
        if long_file["Ticker"][ticker] == market_smith_file["Symbol"][ticker]:

            # Replacing Data
            long_file["ATR"] = market_smith_file["Avg True Range"]
            long_file["Ind Rank"] = market_smith_file["Ind Group Rank"]
            long_file["EPS QoQ"] = market_smith_file["EPS % Chg Last Qtr"]
            long_file["EPS Prev Qtr"] = market_smith_file["EPS % Chg 1 Q Ago"]
            long_file["Sales QoQ"] = market_smith_file["Sales % Chg Lst Qtr"]
            long_file["IBD RS"] = market_smith_file["RS Rating"]
            long_file["C-Price"] = market_smith_file["Current Price"]
            long_file["Earnings Date"] = market_smith_file["EPS Due Date"]
            long_file["50-Day Avg Vol (1000s)"] = market_smith_file["50-Day Avg Vol (1000s)"]
            long_file["Volume (1000s)"] = market_smith_file["Volume (1000s)"]
            long_file["Price % Chg"] = market_smith_file["Price % Chg"]
            long_file["% of Volume"] = round(market_smith_file["Volume (1000s)"] /
                                             market_smith_file["50-Day Avg Vol (1000s)"], 2)
            long_file["% of BO"] = round(long_file["C-Price"] / long_file["BO High"], 2)
            long_file["Sector"] = market_smith_file["Sector"]
            long_file["Industry Name"] = market_smith_file["Industry Name"]

            # Removing existing data
            long_file["Resistance (>20%)"] = ["" for i in range(len(long_file["Ticker"]))]
            long_file["Plurality (TS)"] = ["" for n in range(len(long_file["Ticker"]))]
            long_file["3 Digit"] = ["" for o in range(len(long_file["Ticker"]))]
            long_file["WAM RS"] = ["" for p in range(len(long_file["Ticker"]))]
            long_file["Technical Strength"] = ["" for q in range(len(long_file["Ticker"]))]
            long_file["Price Within 30%"] = ["" for r in range(len(long_file["Ticker"]))]

            # Handling empty cells
            long_file.loc[long_file["EPS QoQ"] == "-", "EPS QoQ"] = 0
            long_file.loc[long_file["EPS Prev Qtr"] == "-", "EPS Prev Qtr"] = 0
            long_file.loc[long_file["Sales QoQ"] == "-", "Sales QoQ"] = 0
            
            # Runner Logic
            comment_list = long_file['Comment'].to_list()
            c_price_list = long_file['C-Price'].to_list()
            final_alert_price = long_file['Final Alert'].to_list()
            bo = []
            for tick in range(len(long_file['Ticker'])):
                if comment_list[tick] == 'Runner':
                    bo.append('Runner')
                elif comment_list[tick] == 'Basing':
                    if c_price_list[tick] > final_alert_price[tick]:
                        bo.append('Runner-Check')
                    else:
                        bo.append('Basing')
                else:
                    bo.append('Check')
            long_file['Comment'] = bo

            # 3-digit logic
            triple_digit = []
            for tri in range(len(long_file["Ticker"])):
                if long_file["Industry Name"][tri] == "Finance-ETF / ETN":
                    triple_digit.append("Yes")
                elif long_file["EPS QoQ"][tri] >= 100:
                    triple_digit.append("Yes")
                elif long_file["Sales QoQ"][tri] >= 100:
                    triple_digit.append("Yes")
                elif long_file["Ind Rank"][tri] <= 20 and long_file["EPS QoQ"][tri] >= 70:
                    triple_digit.append("Yes")
                elif long_file["Ind Rank"][tri] <= 20 and long_file["Sales QoQ"][tri] >= 70:
                    triple_digit.append("Yes")
                elif long_file["EPS QoQ"][tri] >= 70 and long_file["EPS Prev Qtr"][tri] >= 70:
                    triple_digit.append("Yes")
                else:
                    triple_digit.append("")

            long_file["3 Digit"] = triple_digit

            # <30% logic
            bo_high = []
            for price in range(len(long_file["Ticker"])):
                if 1 <= (long_file["C-Price"][price] / long_file["BO High"][price]) <= 1.3:
                    bo_high.append("Yes")
                else:
                    bo_high.append("")

            long_file["Price Within 30%"] = bo_high

            long_file.to_excel(f"C:/Users/krish/Trading Dropbox/Krishna Chaitanya/Process/Stalking/DailyStalking/"
                               f"2024/05.May/Long/{new_stalking_file}.xlsx", index=0)

        else:
            print("Tickers not sorted alphabetically")


In [3]:
long_stalking(old_stalking_file="05-09", marketsmith_file="StalkingLong - 2024-05-10T123744.584",
              new_stalking_file="05-09")